# Import data
Training data

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from utils import get_train_data_only
df_teams, df_BLUE, df_RED = get_train_data_only()

In [3]:
#How many data points we have
len(df_teams), len(df_BLUE), len(df_RED)

(8790, 43950, 43950)

In [4]:
#np.unique(df_teams.patch)

## Rebuild the data into a df that simulates the one we will receive from the web


In [5]:
import numpy as np 
import pandas as pd
from tqdm import tqdm

In [6]:
def rearrange_df(df):
    df_top = df[['champion_id', 'game_id']][df.role == 'TOP']
    df_jgl = df[['champion_id', 'game_id']][df.role == 'JGL']
    df_top_jgl = pd.merge(df_top.rename(columns={'champion_id': 'TOP'}), df_jgl.rename(columns={'champion_id': 'JGL'}), on='game_id')

    df_bot = df[['champion_id', 'game_id']][df.role == 'BOT']
    df_mid = df[['champion_id', 'game_id']][df.role == 'MID']
    df_bot_mid = pd.merge(df_bot.rename(columns={'champion_id': 'BOT'}), df_mid.rename(columns={'champion_id': 'MID'}), on='game_id')

    df_top_jgl_bot_mid = pd.merge(df_top_jgl, df_bot_mid, on='game_id')

    df_sup = df[['champion_id', 'game_id']][df.role == 'SUP']
    df_top_jgl_bot_mid_sup = pd.merge(df_top_jgl_bot_mid, df_sup.rename(columns={'champion_id': 'SUP'}), on='game_id')
    return df_top_jgl_bot_mid_sup

In [7]:
df_blue = rearrange_df(df_BLUE)
df_red = rearrange_df(df_RED)
data = pd.merge(df_blue, df_red, on='game_id')
data

,TOP_x,game_id,JGL_x,BOT_x,MID_x,SUP_x,TOP_y,JGL_y,BOT_y,MID_y,SUP_y
0,69,Neosurf Cup 2020/Scoreboards_1_1,64,21,236,111,68,421,134,39,89
1,516,Random Cup 2020/Scoreboards_1_2,48,498,268,875,82,30,110,80,111
2,240,Random Cup 2020/Scoreboards_1_1,104,236,69,201,516,48,21,7,43
3,62,Underdogs 2020/Scoreboards/Day 5-6_11_1,60,115,246,875,516,104,523,42,412
4,150,Underdogs 2020/Scoreboards/Day 5-6_12_1,203,16,69,12,2,64,15,42,497
...,...,...,...,...,...,...,...,...,...,...,...
8785,31,CBLOL/2020 Season/Split 1/Scoreboards/Week 9_6_1,59,523,69,875,58,60,235,38,412
8786,126,CBLOL/2020 Season/Split 1/Scoreboards/Week 9_7_1,59,110,245,412,516,48,21,134,111
8787,58,CBLOL/2020 Season/Split 1/Scoreboards/Week 9_8_1,79,81,7,350,85,48,21,42,106
8788,58,Brazilian Challenger Circuit/2020 Season/Split...,64,112,517,555,240,113,157,115,497


In [8]:
#side that won
df_teams['game_id'] = df_teams['id']
df_teams[['winner', 'game_id']]

,winner,game_id
17926,BLUE,Neosurf Cup 2020/Scoreboards_1_1
21547,BLUE,Random Cup 2020/Scoreboards_1_2
21546,RED,Random Cup 2020/Scoreboards_1_1
21744,RED,Underdogs 2020/Scoreboards/Day 5-6_11_1
21746,RED,Underdogs 2020/Scoreboards/Day 5-6_12_1
...,...,...
21277,RED,CBLOL/2020 Season/Split 1/Scoreboards/Week 9_6_1
21279,RED,CBLOL/2020 Season/Split 1/Scoreboards/Week 9_7_1
21281,BLUE,CBLOL/2020 Season/Split 1/Scoreboards/Week 9_8_1
21458,RED,Brazilian Challenger Circuit/2020 Season/Split...


In [9]:
full_data = pd.merge(data, df_teams[['winner', 'game_id']], on='game_id')
full_data

,TOP_x,game_id,JGL_x,BOT_x,MID_x,SUP_x,TOP_y,JGL_y,BOT_y,MID_y,SUP_y,winner
0,69,Neosurf Cup 2020/Scoreboards_1_1,64,21,236,111,68,421,134,39,89,BLUE
1,516,Random Cup 2020/Scoreboards_1_2,48,498,268,875,82,30,110,80,111,BLUE
2,240,Random Cup 2020/Scoreboards_1_1,104,236,69,201,516,48,21,7,43,RED
3,62,Underdogs 2020/Scoreboards/Day 5-6_11_1,60,115,246,875,516,104,523,42,412,RED
4,150,Underdogs 2020/Scoreboards/Day 5-6_12_1,203,16,69,12,2,64,15,42,497,RED
...,...,...,...,...,...,...,...,...,...,...,...,...
8785,31,CBLOL/2020 Season/Split 1/Scoreboards/Week 9_6_1,59,523,69,875,58,60,235,38,412,RED
8786,126,CBLOL/2020 Season/Split 1/Scoreboards/Week 9_7_1,59,110,245,412,516,48,21,134,111,RED
8787,58,CBLOL/2020 Season/Split 1/Scoreboards/Week 9_8_1,79,81,7,350,85,48,21,42,106,BLUE
8788,58,Brazilian Challenger Circuit/2020 Season/Split...,64,112,517,555,240,113,157,115,497,RED


## Feature 1: Synergy between champions of the same team (rate of victory for a pair of champions playing together)

In [10]:
'''Rerun this code to update the synergy matrix'''
from build_rate_matrix import synergy_matrix
champions_won_percentage_imputed = synergy_matrix(df_BLUE, df_RED)

100%|██████████████████████████████████████████████████████████████| 152/152 [00:30<00:00,  4.94it/s]


In [11]:
def get_synergy(x,y, df_synergy_matrix):
    try:
        x = df_synergy_matrix.loc[x][y]
    except KeyError:
        x = 0.5
    return x

In [12]:
champions_won_percentage_imputed = pd.read_csv('champions_won_percentage_imputed.csv')
champions_won_percentage_imputed.index = champions_won_percentage_imputed['index']
champions_won_percentage_imputed.drop(['index'], axis=1, inplace=True)

In [13]:
df_synergy = full_data.loc[:,['TOP_x', 'JGL_x','BOT_x','MID_x','SUP_x']]
df_test = df_synergy.apply(lambda z: [get_synergy(x,str(y), champions_won_percentage_imputed) for x in z for y in z if x != y], axis=1).apply(np.mean)
df_test

0       0.510076
1       0.491188
2       0.493680
3       0.490697
4       0.460358
          ...   
8785    0.509530
8786    0.487247
8787    0.503768
8788    0.471324
8789    0.500104
Length: 8790, dtype: float64

In [14]:
from transformers import SynergyFeature

synergy = SynergyFeature()
pd.DataFrame(synergy.fit().transform(df_synergy)).head()

,0
0,0.510076
1,0.491188
2,0.493680
3,0.490697
4,0.460358


## Feature 2: Win rate of champions against the same ROLE of the other team's champion

In [15]:
'''Rerun this code to update the role winrate dataframe'''
from build_rate_matrix import Role_DataFrame
df_role = Role_DataFrame(df_BLUE, df_RED)
df_role

0
champion_id_x role_x role_y champion_id_y win_x     
1             MID    BOT    21            False  1.0
                            235           True   1.0
                            429           False  1.0
                            518           False  1.0
                     JGL    2             False  1.0
...                                              ...
876           TOP    SUP    111           True   1.0
                            432           False  1.0
                     TOP    58            False  1.0
                            126           False  0.5
                                          True   0.5

[42354 rows x 1 columns]

In [16]:
full_data

,TOP_x,game_id,JGL_x,BOT_x,MID_x,SUP_x,TOP_y,JGL_y,BOT_y,MID_y,SUP_y,winner
0,69,Neosurf Cup 2020/Scoreboards_1_1,64,21,236,111,68,421,134,39,89,BLUE
1,516,Random Cup 2020/Scoreboards_1_2,48,498,268,875,82,30,110,80,111,BLUE
2,240,Random Cup 2020/Scoreboards_1_1,104,236,69,201,516,48,21,7,43,RED
3,62,Underdogs 2020/Scoreboards/Day 5-6_11_1,60,115,246,875,516,104,523,42,412,RED
4,150,Underdogs 2020/Scoreboards/Day 5-6_12_1,203,16,69,12,2,64,15,42,497,RED
...,...,...,...,...,...,...,...,...,...,...,...,...
8785,31,CBLOL/2020 Season/Split 1/Scoreboards/Week 9_6_1,59,523,69,875,58,60,235,38,412,RED
8786,126,CBLOL/2020 Season/Split 1/Scoreboards/Week 9_7_1,59,110,245,412,516,48,21,134,111,RED
8787,58,CBLOL/2020 Season/Split 1/Scoreboards/Week 9_8_1,79,81,7,350,85,48,21,42,106,BLUE
8788,58,Brazilian Challenger Circuit/2020 Season/Split...,64,112,517,555,240,113,157,115,497,RED


In [17]:
rate_champion_vs_champion = pd.read_csv('role_winrate_champ_vs_champ.csv',index_col=[0,1,2,3,4])
rate_champion_vs_champion

0
champion_id_x role_x role_y champion_id_y win_x     
1             MID    BOT    21            False  1.0
                            235           True   1.0
                            429           False  1.0
                            518           False  1.0
                     JGL    2             False  1.0
...                                              ...
876           TOP    SUP    111           True   1.0
                            432           False  1.0
                     TOP    58            False  1.0
                            126           False  0.5
                                          True   0.5

[42354 rows x 1 columns]

In [18]:
def get_vs_rate(id_x, role, id_y, rate_champion_vs_champion):
    try:
        x = rate_champion_vs_champion.loc[id_x, role, role, id_y, True]
    except KeyError:
        try:
            x = 1-rate_champion_vs_champion.loc[id_x, role, role, id_y, False]
        except KeyError:
            x = 0.5
    return x

In [19]:
role='TOP'
column_1, column_2 = 'TOP_x', 'TOP_y'
role_data = full_data[['TOP_x', 'TOP_y']] 
role_data.apply(lambda z: get_vs_rate(z[0], role, z[1], rate_champion_vs_champion), axis=1)

,0
0,1.000000
1,0.601626
2,0.000000
3,0.454545
4,0.000000
...,...
8785,0.200000
8786,0.592593
8787,0.500000
8788,0.571429


# Machine Learning

## Pipe with logit regression

In [20]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
#from sklearn.compose import make_column_transformer
from sklearn.compose import ColumnTransformer
from sklearn.compose import make_column_selector
from sklearn import set_config; set_config(display='diagram')

from transformers import SynergyFeature, RoleFeature

# Paralellize column transformers
preproc = ColumnTransformer([
    ('synergy_blue', SynergyFeature(), ['TOP_x', 'JGL_x','BOT_x','MID_x','SUP_x']),
    ('synergy_red', SynergyFeature(), ['TOP_y', 'JGL_y','BOT_y','MID_y','SUP_y']),
    ('TOP_rate', RoleFeature('TOP'), ['TOP_x', 'TOP_y']),
    ('SUP_rate', RoleFeature('SUP'), ['SUP_x', 'SUP_y']),
    ('MID_rate', RoleFeature('MID'), ['MID_x', 'MID_y']),
    ('BOT_rate', RoleFeature('BOT'), ['BOT_x', 'BOT_y']),
    ('JGL_rate', RoleFeature('JGL'), ['JGL_x', 'JGL_y'])
    ])

#add model
pipe = make_pipeline(preproc, LogisticRegression(solver='liblinear'))
pipe

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('synergy_blue',
                                                  SynergyFeature(),
                                                  ['TOP_x', 'JGL_x', 'BOT_x',
                                                   'MID_x', 'SUP_x']),
                                                 ('synergy_red',
                                                  SynergyFeature(),
                                                  ['TOP_y', 'JGL_y', 'BOT_y',
                                                   'MID_y', 'SUP_y']),
                                                 ('TOP_rate',
                                                  RoleFeature(role='TOP'),
                                                  ['TOP_x', 'TOP_y']),
                                                 ('SUP_rate',
                                                  RoleFeature(role='SUP'),
                                                  ['SUP_x', 'SUP_y']),
                                                 ('MID_rate',
                                                  RoleFeature(role='MID'),
                                                  ['MID_x', 'MID_y']),
                                                 ('BOT_rate',
                                                  RoleFeature(role='BOT'),
                                                  ['BOT_x', 'BOT_y']),
                                                 ('JGL_rate',
                                                  RoleFeature(role='JGL'),
                                                  ['JGL_x', 'JGL_y'])])),
                ('logisticregression', LogisticRegression(solver='liblinear'))])

In [21]:
full_data

,TOP_x,game_id,JGL_x,BOT_x,MID_x,SUP_x,TOP_y,JGL_y,BOT_y,MID_y,SUP_y,winner
0,69,Neosurf Cup 2020/Scoreboards_1_1,64,21,236,111,68,421,134,39,89,BLUE
1,516,Random Cup 2020/Scoreboards_1_2,48,498,268,875,82,30,110,80,111,BLUE
2,240,Random Cup 2020/Scoreboards_1_1,104,236,69,201,516,48,21,7,43,RED
3,62,Underdogs 2020/Scoreboards/Day 5-6_11_1,60,115,246,875,516,104,523,42,412,RED
4,150,Underdogs 2020/Scoreboards/Day 5-6_12_1,203,16,69,12,2,64,15,42,497,RED
...,...,...,...,...,...,...,...,...,...,...,...,...
8785,31,CBLOL/2020 Season/Split 1/Scoreboards/Week 9_6_1,59,523,69,875,58,60,235,38,412,RED
8786,126,CBLOL/2020 Season/Split 1/Scoreboards/Week 9_7_1,59,110,245,412,516,48,21,134,111,RED
8787,58,CBLOL/2020 Season/Split 1/Scoreboards/Week 9_8_1,79,81,7,350,85,48,21,42,106,BLUE
8788,58,Brazilian Challenger Circuit/2020 Season/Split...,64,112,517,555,240,113,157,115,497,RED


In [22]:
from sklearn.preprocessing import LabelEncoder

y_train = LabelEncoder().fit(full_data.winner).transform(full_data.winner)
X_train = full_data.drop(['game_id', 'winner'], axis=1)

In [23]:
# Train pipeline
pipe.fit(X_train,y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('synergy_blue',
                                                  SynergyFeature(),
                                                  ['TOP_x', 'JGL_x', 'BOT_x',
                                                   'MID_x', 'SUP_x']),
                                                 ('synergy_red',
                                                  SynergyFeature(),
                                                  ['TOP_y', 'JGL_y', 'BOT_y',
                                                   'MID_y', 'SUP_y']),
                                                 ('TOP_rate',
                                                  RoleFeature(role='TOP'),
                                                  ['TOP_x', 'TOP_y']),
                                                 ('SUP_rate',
                                                  RoleFeature(role='SUP'),
                                                  ['SUP_x', 'SUP_y']),
                                                 ('MID_rate',
                                                  RoleFeature(role='MID'),
                                                  ['MID_x', 'MID_y']),
                                                 ('BOT_rate',
                                                  RoleFeature(role='BOT'),
                                                  ['BOT_x', 'BOT_y']),
                                                 ('JGL_rate',
                                                  RoleFeature(role='JGL'),
                                                  ['JGL_x', 'JGL_y'])])),
                ('logisticregression', LogisticRegression(solver='liblinear'))])

In [24]:
from sklearn.model_selection import cross_val_score

# Cross validate pipeline
cross_val_score(pipe, X_train, y_train, cv=2, scoring='accuracy').mean()

0.8113765642775881

In [25]:
import joblib

joblib.dump(pipe, 'model.joblib')

['model.joblib']

# Use the test data

## Import the test data

In [26]:
#Import the data through the utils file
test_teams, test_BLUE, test_RED = get_train_data_only(train_data = False, test_data = True)
len(test_teams), len(test_BLUE), len(test_RED)

(27, 135, 135)

In [27]:
#Fix the dataframe with the columns we need
test_blue = rearrange_df(test_BLUE)
test_red = rearrange_df(test_RED)
test_data = pd.merge(test_blue, test_red, on='game_id')
test_data

,TOP_x,game_id,JGL_x,BOT_x,MID_x,SUP_x,TOP_y,JGL_y,BOT_y,MID_y,SUP_y
0,13,Liga Tica de Leyendas/2020 Season/Scoreboards/...,59,81,103,201,82,421,134,236,875
1,516,Liga Tica de Leyendas/2020 Season/Scoreboards/...,2,81,68,350,164,60,21,3,412
2,6,Liga Tica de Leyendas/2020 Season/Scoreboards/...,104,429,61,111,266,113,51,8,43
3,85,Liga Tica de Leyendas/2020 Season/Scoreboards/...,59,523,69,89,875,113,498,112,497
4,516,Liga Tica de Leyendas/2020 Season/Scoreboards/...,48,236,90,201,84,79,429,518,432
5,82,Liga Tica de Leyendas/2020 Season/Scoreboards/...,48,236,42,111,57,5,119,134,497
6,875,Liga Tica de Leyendas/2020 Season/Scoreboards/...,245,18,84,111,69,59,110,3,201
7,31,Liga Nexo/2020 Season/Split 1/Scoreboards/Week...,104,145,61,412,98,107,81,112,350
8,57,Liga Nexo/2020 Season/Split 1/Scoreboards/Week...,104,22,7,223,58,48,523,517,26
9,24,Liga Nexo/2020 Season/Split 1/Scoreboards/Week...,104,134,57,44,41,30,21,516,143


In [28]:
#get the side that won
test_teams['game_id'] = test_teams['id']
test_teams[['winner', 'game_id']]

,winner,game_id
21605,BLUE,Liga Tica de Leyendas/2020 Season/Scoreboards/...
21603,RED,Liga Tica de Leyendas/2020 Season/Scoreboards/...
21604,BLUE,Liga Tica de Leyendas/2020 Season/Scoreboards/...
21597,RED,Liga Tica de Leyendas/2020 Season/Scoreboards/...
21596,BLUE,Liga Tica de Leyendas/2020 Season/Scoreboards/...
21595,RED,Liga Tica de Leyendas/2020 Season/Scoreboards/...
21594,BLUE,Liga Tica de Leyendas/2020 Season/Scoreboards/...
21574,RED,Liga Nexo/2020 Season/Split 1/Scoreboards/Week...
21572,RED,Liga Nexo/2020 Season/Split 1/Scoreboards/Week...
21567,BLUE,Liga Nexo/2020 Season/Split 1/Scoreboards/Week...


In [29]:
full_test_data = pd.merge(test_data, test_teams[['winner', 'game_id']], on='game_id')
full_test_data

,TOP_x,game_id,JGL_x,BOT_x,MID_x,SUP_x,TOP_y,JGL_y,BOT_y,MID_y,SUP_y,winner
0,13,Liga Tica de Leyendas/2020 Season/Scoreboards/...,59,81,103,201,82,421,134,236,875,BLUE
1,516,Liga Tica de Leyendas/2020 Season/Scoreboards/...,2,81,68,350,164,60,21,3,412,RED
2,6,Liga Tica de Leyendas/2020 Season/Scoreboards/...,104,429,61,111,266,113,51,8,43,BLUE
3,85,Liga Tica de Leyendas/2020 Season/Scoreboards/...,59,523,69,89,875,113,498,112,497,RED
4,516,Liga Tica de Leyendas/2020 Season/Scoreboards/...,48,236,90,201,84,79,429,518,432,BLUE
5,82,Liga Tica de Leyendas/2020 Season/Scoreboards/...,48,236,42,111,57,5,119,134,497,RED
6,875,Liga Tica de Leyendas/2020 Season/Scoreboards/...,245,18,84,111,69,59,110,3,201,BLUE
7,31,Liga Nexo/2020 Season/Split 1/Scoreboards/Week...,104,145,61,412,98,107,81,112,350,RED
8,57,Liga Nexo/2020 Season/Split 1/Scoreboards/Week...,104,22,7,223,58,48,523,517,26,RED
9,24,Liga Nexo/2020 Season/Split 1/Scoreboards/Week...,104,134,57,44,41,30,21,516,143,BLUE


In [30]:
np.unique(test_teams.patch)

array([10.9], dtype=float32)

## Test the data

In [31]:
y_test = LabelEncoder().fit(full_test_data.winner).transform(full_test_data.winner)
X_test = full_test_data.drop(['game_id', 'winner'], axis=1)

In [32]:
pipe.score(X_test, y_test)

0.5185185185185185

In [33]:
from sklearn import metrics

predicted = pipe.predict(X_test)
print(metrics.accuracy_score(y_test, predicted))
print(metrics.classification_report(y_test, predicted)) 

0.5185185185185185
              precision    recall  f1-score   support

           0       0.50      0.69      0.58        13
           1       0.56      0.36      0.43        14

    accuracy                           0.52        27
   macro avg       0.53      0.52      0.51        27
weighted avg       0.53      0.52      0.51        27



# Evaluate data

## Import the evaluate data

In [34]:
#Import the data through the utils file
eval_teams, eval_BLUE, eval_RED = get_train_data_only(train_data = False, evaluate_data = True)
len(eval_teams), len(eval_BLUE), len(eval_RED)

(5, 25, 25)

In [35]:
#Teams DataFrame used for Machine Learning
eval_teams_ML = eval_teams[['id', 'patch', 'year', 'winner']]
eval_teams_ML.head(5)

,id,patch,year,winner
31455,Magyar Nemzeti E-sport Bajnokság/Season 3/Scor...,11.9,2021,BLUE
31446,Magyar Nemzeti E-sport Bajnokság/Season 3/Scor...,11.9,2021,RED
31445,Magyar Nemzeti E-sport Bajnokság/Season 3/Scor...,11.9,2021,RED
31502,Magyar Nemzeti E-sport Bajnokság/Season 3/Scor...,11.9,2021,BLUE
31515,2021 Mid-Season Invitational/Scoreboards_5_1,11.9,2021,RED


## Synergy Feature

In [36]:
#Get the synergy of the Blue team champions
eval_blue = pair_wise_synergy(eval_BLUE, champions_won_percentage_imputed, 'blue')
eval_blue['id'] = eval_blue.index
eval_teams_ML =pd.merge(eval_teams_ML, eval_blue, on='id', how='inner')

NameError: name 'pair_wise_synergy' is not defined

In [ ]:
#Get the synergy of the Red team champions
eval_red = pair_wise_synergy(eval_RED, champions_won_percentage_imputed, 'red')
eval_red['id'] = eval_red.index
eval_teams_ML =pd.merge(eval_teams_ML, eval_red, on='id', how='inner')

In [ ]:
eval_teams_ML

## Role rate winrate

In [ ]:
eval_blue_red = pd.merge(left=eval_BLUE, right=eval_RED, left_on= 'game_id', right_on= 'game_id')
eval_role = eval_blue_red[['champion_id_x', 'role_x', 'role_y', 'champion_id_y', 'win_x', 'game_id']]
eval_role.head(2)

In [ ]:
eval_same_role = eval_role[eval_role['role_x'] == eval_role['role_y']]
eval_same_role['same_role_win_rate'] = eval_same_role.apply(lambda z: 
              get_vs_rate(z.champion_id_x, z.role_x, z.role_y, z.champion_id_y),
              axis=1)
eval_same_role.head(5)

In [ ]:
roles = ['TOP', 'JGL', 'MID', 'BOT', 'SUP']

for role in roles:
    eval_role = eval_same_role[eval_same_role.role_x == role][['game_id', 'same_role_win_rate']]
    eval_role[role] = eval_role['same_role_win_rate']
    eval_role['id'] = eval_role['game_id']
    eval_role.drop(['game_id', 'same_role_win_rate'], axis=1, inplace=True)
    eval_teams_ML = pd.merge(eval_teams_ML, eval_role, on='id', how='inner')

eval_teams_ML.head(2)

In [ ]:
y_eval = LabelEncoder().fit(eval_teams_ML.winner).transform(eval_teams_ML.winner)
X_eval = eval_teams_ML.drop(['id', 'winner', 'patch', 'year'], axis=1)

In [ ]:
pipe.predict(X_eval), y_eval